# PPO Breakout Example

### Atari Breakout

Please do note that this example may take a long time to train.

With the default 4 threads runnning on an 8-core CPU with a GTX 1080 Ti, it will take several hours to train to a decent level of play.

Running on a platform with more GPU power and a larger cluster of CPUs could siginificantly reduce training time.

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPool2D, Flatten
from tensorflow.keras.backend import categorical_crossentropy
from ludus.policies import BaseTrainer
from ludus.env import EnvController
from ludus.utils import preprocess_atari, reshape_train_var
import gym
# Super Mario stuff
from nes_py.wrappers import BinarySpaceToDiscreteSpaceEnv
import gym_super_mario_bros
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT

In [2]:
def make_env():
    env = gym_super_mario_bros.make('SuperMarioBros-v0')
    env = BinarySpaceToDiscreteSpaceEnv(env, SIMPLE_MOVEMENT)
    return env

In [3]:
class IMTrainer(BaseTrainer):
    def __init__(self, in_op, out_op, value_out_op, act_type='discrete', sess=None, clip_val=0.2, ppo_iters=80,
                 target_kl=0.01, v_coef=1., entropy_coef=0.01):
        self.value_out_op = value_out_op
        self.clip_val = clip_val
        self.ppo_iters = ppo_iters
        self.target_kl = target_kl
        self.v_coef = v_coef
        self.entropy_coef = entropy_coef
        
        # ICM parameters, TODO: make parameters for these
        self.ro_coef = 0.1
        self.beta = 0.2
        self.eta = 1
        self.r_i_coef = 1
        self.r_e_coef = 0.05
        
        super().__init__(in_op, out_op, act_type, sess)
        
    def _create_ICM(self, optimizer=tf.train.AdamOptimizer()):
        feature_dim = 64 # TODO: make a parameter for this
        
        # Create placeholder
        self.next_obs_holders = tf.placeholder(tf.float32, shape=self.in_op.shape)
        
        # Observation feature encoder
        with tf.variable_scope('feature_encoder'):
            enc_layers = [
                Conv2D(16, 4, (2, 2), activation=tf.nn.relu, name='fe_conv'),
                MaxPool2D(2, name='fe_max_pool'),
                Conv2D(16, 3, activation=tf.nn.relu, name='fe_conv2'),
                MaxPool2D(2, name='fe_max_pool2'),
                Conv2D(32, 3, activation=tf.nn.relu, name='fe_conv3'),
                MaxPool2D(2, name='fe_max_pool3'),
                Flatten(name='fe_flattened'),
                Dense(feature_dim, activation=tf.nn.relu, name='fe_dense')
            ]
            
            # Encoding state
            self.f_obs = enc_layers[0](self.in_op)
            for i in range(1, len(enc_layers)):
                self.f_obs = enc_layers[i](self.f_obs)
            
            # Encoding the next state
            self.f_obs_next = enc_layers[0](self.next_obs_holders)
            for i in range(1, len(enc_layers)):
                self.f_obs_next = enc_layers[i](self.f_obs_next)
            
        # State predictor forward model
        with tf.variable_scope('forward_model'):
            self.state_act_pair = tf.concat([self.out_op, self.f_obs], axis=1)
            self.sp_dense = Dense(64, activation=tf.nn.relu)(self.state_act_pair)
            self.f_obs_next_pred = Dense(feature_dim, activation=tf.nn.relu)(self.sp_dense)
        
        # Inverse dynamics model (predicting action)
        with tf.variable_scope('inverse_model'):
            self.state_state_pair = tf.concat([self.f_obs, self.f_obs_next], axis=1)
            self.act_preds = Dense(64, activation=tf.nn.relu)(self.state_state_pair)
            # TODO: softmax only works for discrete
            self.act_preds = Dense(self.out_op.shape[1], activation=tf.nn.softmax)(self.act_preds)
        
        # Calculating intrinsic reward
        self.obs_pred_diff = self.f_obs_next_pred - self.f_obs_next
        self.r_i = 0.5 * self.eta * tf.reduce_sum(self.obs_pred_diff ** 2, axis=1) # Fix these squares (Probably okay)
        self.r_ie = self.r_i_coef * self.r_i + self.r_e_coef * self.reward_holders
        
        # Calculating losses
        self.pre_loss_i = categorical_crossentropy(self.act_masks, self.act_preds) # tf.reduce_sum((self.act_holders - self.act_pred) ** 2, axis=1)
        self.pre_loss_f = 0.5 * tf.reduce_sum(self.obs_pred_diff ** 2, axis=1)
        
        self.loss_i = (1 - self.beta) * tf.reduce_mean(self.pre_loss_i)
        self.loss_f = self.beta * tf.reduce_mean(self.pre_loss_f)
        self.loss_p = -self.ro_coef * tf.reduce_sum(self.r_ie)
        
        # Making update functions
        self.i_train_vars = tf.trainable_variables(scope='feature_encoder') + tf.trainable_variables(scope='inverse_model')
        self.f_train_vars = tf.trainable_variables(scope='forward_model')
        self.p_train_vars = [var for var in tf.trainable_variables() if var not in (self.i_train_vars + self.f_train_vars)]
        
        self.li_update = optimizer.minimize(self.loss_i, var_list=self.i_train_vars)
        self.lf_update = optimizer.minimize(self.loss_f, var_list=self.f_train_vars)
        self.lp_update = optimizer.minimize(self.loss_p, var_list=self.p_train_vars)
        
        self.icm_updates = [self.li_update, self.lf_update, self.lp_update]
        
    def _create_discrete_trainer(self, optimizer=tf.train.AdamOptimizer()):
        """
        Creates a function for vanilla policy training with a discrete action space
        """
        # First passthrough
        
        self.act_holders = tf.placeholder(tf.int32, shape=[None])
        self.reward_holders = tf.placeholder(tf.float32, shape=[None])
        
        self.act_masks = tf.one_hot(self.act_holders, self.out_op.shape[1].value, dtype=tf.float32)
        self.resp_acts = tf.reduce_sum(self.act_masks *  self.out_op, axis=1)
        
        self.advantages = self.reward_holders - tf.squeeze(self.value_out_op)
        
        self._create_ICM()
        
        # Second passthrough
        
        self.advatange_holders = tf.placeholder(dtype=tf.float32, shape=self.advantages.shape)
        self.old_prob_holders = tf.placeholder(dtype=tf.float32, shape=self.resp_acts.shape)
 
        self.policy_ratio = self.resp_acts / self.old_prob_holders
        self.clipped_ratio = tf.clip_by_value(self.policy_ratio, 1 - self.clip_val, 1 + self.clip_val)

        self.min_loss = tf.minimum(self.policy_ratio * self.advatange_holders, self.clipped_ratio * self.advatange_holders)
        
        self.optimizer = tf.train.AdamOptimizer()

        # Actor update
        
        self.kl_divergence = tf.reduce_mean(tf.log(self.old_prob_holders) - tf.log(self.resp_acts))
        self.actor_loss = -tf.reduce_mean(self.min_loss)
        self.actor_update = self.optimizer.minimize(self.actor_loss)

        # Value update
        
        self.value_loss = tf.reduce_mean(tf.square(self.reward_holders - tf.squeeze(self.value_out_op)))
        self.value_update = self.optimizer.minimize(self.value_loss)
        
        # Intrinsic motivation update
        
        self.intrinsic_loss = self.loss_i + self.loss_f
        self.intrinsic_update = self.optimizer.minimize(self.intrinsic_loss)
        
        # Combined update
        
        self.entropy = -tf.reduce_mean(tf.reduce_sum(self.out_op * tf.log(1. / tf.clip_by_value(self.out_op, 1e-8, 1.0)), axis=1))
        self.combined_loss = self.actor_loss + self.v_coef * self.value_loss + self.entropy_coef * self.entropy
        self.combined_update = self.optimizer.minimize(self.combined_loss)
        
        def update_func(train_data):
            i_rew, _, _, _ = self.sess.run([self.r_i] + self.icm_updates, 
                                   feed_dict={self.in_op: reshape_train_var(train_data[:, 0]),
                                              self.act_holders: reshape_train_var(train_data[:, 1]),
                                              self.reward_holders: train_data[:, 2],
                                              self.next_obs_holders: reshape_train_var(train_data[:, 3])})
            
#             self.old_probs, self.old_advantages = self.sess.run([self.resp_acts, self.advantages], 
#                                     feed_dict={self.in_op: reshape_train_var(train_data[:, 0]),
#                                                self.act_holders: train_data[:, 1],
#                                                self.reward_holders: train_data[:, 2]})
    
#             for i in range(self.ppo_iters):
#                 kl_div, i_rew, _ = self.sess.run([self.kl_divergence, self.r_i, self.intrinsic_loss], 
#                                    feed_dict={self.in_op: reshape_train_var(train_data[:, 0]),
#                                         self.act_holders: reshape_train_var(train_data[:, 1]),
#                                         self.reward_holders: train_data[:, 2],
#                                         self.old_prob_holders: self.old_probs,
#                                         self.advatange_holders: self.old_advantages})
#                 i_rews.append(i_rew)
#                 if kl_div > 1.5 * self.target_kl:
#                     break    
    
            return i_rew

        self.sess.run(tf.global_variables_initializer())
        
        return update_func
        
    def _create_continuous_trainer(self):
        return

In [4]:
env = make_env() # This instance of the environment is only used
                              # to get action dimensions
in_shape = [84, 84, 1] # Size of reshaped observations

# Creating a conv net for the policy and value estimator
obs_op = Input(shape=in_shape)
conv1 = Conv2D(16, 8, (4, 4), activation='relu')(obs_op)
max_pool1 = MaxPool2D(2, 2)(conv1)
conv2 = Conv2D(32, 4, (2, 2), activation='relu')(max_pool1)
max_pool2 = MaxPool2D(2, 2)(conv2)
dense1 = Dense(256, activation='relu')(max_pool2)
flattened = Flatten()(dense1)

# Output probability distribution over possible actions
act_probs_op = Dense(env.action_space.n, activation='softmax')(flattened)

# Output value of observed state
value_op = Dense(1)(flattened)

# Wrap a Proximal Policy Optimization Trainer on top of the network
network = IMTrainer(obs_op, act_probs_op, value_op, act_type='discrete', ppo_iters=80)

/home/ejmejm/anaconda3/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


In [ ]:
n_episodes = 10000 # Total episodes of data to collect
max_steps = 2048 # Max number of frames per game
batch_size = 4 # Smaller = faster, larger = stabler
print_freq = 1 # How many training updates between printing progress

In [ ]:
# Create the environment controller for generating game data
ec = EnvController(make_env, n_threads=4)
# Set the preprocessing function for observations
ec.set_obs_transform(lambda x: preprocess_atari(x.squeeze()))

In [ ]:
update_rewards = []
update_i_rewards = []

for i in range(int(n_episodes / batch_size)):
    ec.sim_episodes(network, batch_size, max_steps) # Simualate env to generate data
    update_rewards.append(ec.get_avg_reward()) # Append rewards to reward tracker list
    dat = ec.get_data() # Get all the data gathered
    i_rew = network.train(dat) # Train the network with PPO
    update_i_rewards.append(i_rew)
    if i != 0 and i % print_freq == 0:
        print(f'Update #{i}, Avg Reward (E, I): {np.mean(update_rewards[-print_freq:])}, {np.mean(update_i_rewards[-print_freq:])}') # Print an update
    if i != 0 and i % (print_freq * 5) == 0:
        ec.render_episodes(network, 5, max_steps)

Update #1, Avg Reward (E, I): 570.5, 0.025459524244070053
Update #2, Avg Reward (E, I): 481.75, 0.019870921969413757
Update #3, Avg Reward (E, I): 571.5, 0.017639750614762306
Update #4, Avg Reward (E, I): 448.5, 0.018466830253601074
Update #5, Avg Reward (E, I): 509.0, 0.02011704072356224
Update #6, Avg Reward (E, I): 530.5, 0.02301792800426483
Update #7, Avg Reward (E, I): 445.75, 0.02790858969092369
Update #8, Avg Reward (E, I): 407.0, 0.038784295320510864
Update #9, Avg Reward (E, I): 257.75, 0.060099150985479355
Update #10, Avg Reward (E, I): 189.25, 0.09475848823785782
Update #11, Avg Reward (E, I): 361.25, 0.15753334760665894
Update #12, Avg Reward (E, I): 432.0, 0.24771468341350555
Update #13, Avg Reward (E, I): 369.5, 0.4025205969810486
Update #14, Avg Reward (E, I): 449.0, 0.625564694404602
Update #15, Avg Reward (E, I): 447.25, 0.9790916442871094
Update #16, Avg Reward (E, I): 576.5, 1.472145915031433
Update #17, Avg Reward (E, I): 480.25, 2.0971384048461914
Update #18, Avg R

In [ ]:
ec.render_episodes(network, 5, max_steps) # Render an episode to see the result